In [1]:
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
import os
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pyspark import SparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql.functions import avg, pow, col, sqrt, lit
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from splearn.rdd import ArrayRDD
from splearn.preprocessing import SparkLabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from splearn import BlockRDD
from splearn import SparseRDD,DictRDD
from splearn.utils.validation import check_rdd
from splearn.linear_model import SparkLinearRegression

In [2]:
data = sc.textFile('../dac/train.txt', 10000)
sample = sc.textFile('../dac_sample/dac_sample.txt', use_unicode=False).map(lambda x: x.split('\t'))
label_sample = sample.map(lambda x:x[0])
sample = sample.map(lambda x: x[1:])
split = data.map(lambda x: x.split('\t'))
label = split.map(lambda x: x[0])
entire_train = split.map(lambda x: x[1:])

In [3]:
train10M, train38M = entire_train.randomSplit([10,38])
train5M, validation2M, test3M = train10M.randomSplit([5,2,3])

In [4]:
df_sample = sqlContext.createDataFrame(sample, ['C'+str(i) for i in range(39)])

#Fill in Numeric Missing Value with Mean
1) fill in median for numeric data

In [5]:
def fillMissing(df):
    new = df.fillna(0)
    new = new.na.replace('','NA')
    return new

In [6]:
filled_sample = fillMissing(df_sample)

#Numeric Column Type Change

In [7]:
def changeDataType(df):
    new = df    
    for i in range(13):
        new = new.withColumn('C'+str(i), new['C'+str(i)].cast('int'))
    return new

In [8]:
changed_type_sample = changeDataType(filled_sample)

#Histogram

In [9]:
def drawHistogram(cols, i):
    col = cols[i]
    count = col.histogram(col.distinct().count())
    return count

In [ ]:
print "drawing histogram"
for i in range(13):
    print str(i) + "th col"
    new = drawHistogram(cols, i)
    plt.bar(new[0][1:], np.log10(new[1]), width = 1)
    plt.savefig("../images/log_col_"+str(i)+".png")

#Standardization

In [10]:
def standardize(df):
    cols = df.columns[:13]
    for i,each in enumerate(cols):
        print "start" + each
        mean  = df.select(avg(each)).collect()[0][0]
        std = df.select(sqrt(avg(col(each) * col(each)) - pow(avg(col(each)), lit(2)))).collect()[0][0]
        df = df.withColumn(each, (col(each)-mean/std))
    return df
        

In [11]:
standardize_sample  = standardize(changed_type_sample)

startC0
startC1
startC2
startC3
startC4
startC5
startC6
startC7
startC8
startC9
startC10
startC11
startC12


In [12]:
standardize_sample

DataFrame[C0: double, C1: double, C2: double, C3: double, C4: double, C5: double, C6: double, C7: double, C8: double, C9: double, C10: double, C11: double, C12: double, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string]

#LabelEncoding

In [13]:
def labelencoder(df):
    cols = df.columns[13:]
    for i,each in enumerate(cols):
        print "start"+each
        indexer = StringIndexer(inputCol= each, outputCol = "indexed"+each)
        model = indexer.fit(df)
        indexed = model.transform(df)
        indexed  = indexed.drop(each)
        df = indexed
    return indexed

In [14]:
indexed_sample = labelencoder(standardize_sample)

startC13
startC14
startC15
startC16
startC17
startC18
startC19
startC20
startC21
startC22
startC23
startC24
startC25
startC26
startC27
startC28
startC29
startC30
startC31
startC32
startC33
startC34
startC35
startC36
startC37
startC38


#Feature Dropping

In [181]:
indexed_sample.describe('C4').show()

+-------+-----+
|summary|   C4|
+-------+-----+
|  count|95240|
|   mean|  NaN|
| stddev|  NaN|
|    min|  NaN|
|    max|  NaN|
+-------+-----+



In [202]:
indexed_sample.describe('indexedC24').show()

+-------+------------------+
|summary|        indexedC24|
+-------+------------------+
|  count|            100000|
|   mean|        8862.44405|
| stddev|12545.962863902858|
|    min|               0.0|
|    max|           41311.0|
+-------+------------------+



In [211]:
indexed_sample.describe('indexedC33').show()

+-------+------------------+
|summary|        indexedC33|
+-------+------------------+
|  count|            100000|
|   mean|        7803.20156|
| stddev|11492.451306542636|
|    min|               0.0|
|    max|           38617.0|
+-------+------------------+



In [216]:
indexed_sample.describe('indexedC38').show()

+-------+------------------+
|summary|        indexedC38|
+-------+------------------+
|  count|            100000|
|   mean|         628.59351|
| stddev|1668.5884641384405|
|    min|               0.0|
|    max|            9526.0|
+-------+------------------+



In [62]:
indexed_sample.describe('C11').show()

+-------+------------------+
|summary|               C11|
+-------+------------------+
|  count|             22820|
|   mean|0.7617504576412591|
| stddev| 5.327551619893342|
|    min|   -0.176023424918|
|    max|  492.823976575082|
+-------+------------------+



#Drop categoric feature which has huge std

In [19]:
indexed_sample = indexed_sample.drop('C4')
indexed_sample = indexed_sample.drop('indexedC15')
indexed_sample = indexed_sample.drop('indexedC24')
indexed_sample = indexed_sample.drop('indexedC28')
indexed_sample = indexed_sample.drop('indexedC33')

#Filtering out rare categories in categorical features

In [20]:
names = ['C'+str(i) for i in range(4)] + ['C'+str(i) for i in range(5,13)]+['indexedC'+str(i) for i in range(13,15)]+['indexedC'+str(i) for i in range(16,24)]+['indexedC'+str(i) for i in range(25,28)]+['indexedC'+str(i) for i in range(29,33)]+['indexedC'+str(i) for i in range(34,39)]

In [21]:
def categoryfilter(df, names):
    #drop one numeric column,
    cols = names[12:]
    for i, each in enumerate(cols):
        print "start replacing"+each
        df.withColumn(each, when(df[each]<10, df[each]).otherwise(11))
    return df
        

In [22]:
filltered_sample = categoryfilter(indexed_sample, names)

start replacingindexedC13
start replacingindexedC14
start replacingindexedC16
start replacingindexedC17
start replacingindexedC18
start replacingindexedC19
start replacingindexedC20
start replacingindexedC21
start replacingindexedC22
start replacingindexedC23
start replacingindexedC25
start replacingindexedC26
start replacingindexedC27
start replacingindexedC29
start replacingindexedC30
start replacingindexedC31
start replacingindexedC32
start replacingindexedC34
start replacingindexedC35
start replacingindexedC36
start replacingindexedC37
start replacingindexedC38


#One-Hot Encoding

In [23]:
def onehotencoder(df, names):
    cols = names[12:]
    for i,each in enumerate(cols):
        print "start"+each
        encoder = OneHotEncoder(dropLast=True, inputCol= each, outputCol ="encoded_"+each)  
        encoded = encoder.transform(df)
        encoded = encoded.drop(each)
        df = encoded
    return encoded

In [24]:
encoded_sample = onehotencoder(filltered_sample, names)

startindexedC13
startindexedC14
startindexedC16
startindexedC17
startindexedC18
startindexedC19
startindexedC20
startindexedC21
startindexedC22
startindexedC23
startindexedC25
startindexedC26
startindexedC27
startindexedC29
startindexedC30
startindexedC31
startindexedC32
startindexedC34
startindexedC35
startindexedC36
startindexedC37
startindexedC38


In [25]:
encoded_sample = encoded_sample.na.fill(0)

#Modeling - Logistic Regression

In [26]:
names = ['C'+str(i) for i in range(4)] + \
        ['C'+str(i) for i in range(5,13)]+ \
        ['encoded_indexedC'+str(i) for i in range(13,15)]+\
        ['encoded_indexedC'+str(i) for i in range(16,24)]+\
        ['encoded_indexedC'+str(i) for i in range(25,28)]+\
        ['encoded_indexedC'+str(i) for i in range(29,33)]+\
        ['encoded_indexedC'+str(i) for i in range(34,39)]
assembler = VectorAssembler(
    inputCols=names,
    outputCol="features")

sample_features = assembler.transform(encoded_sample).select(col('features'))

In [30]:
sample_features[0]
#train_sample= combine.map(lambda x: LabeledPoint(x[0], x[1]))

KeyboardInterrupt: 

In [47]:
lrm = LogisticRegressionWithSGD.train(train_sample, iterations=1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 115.0 failed 1 times, most recent failure: Lost task 0.0 in stage 115.0 (TID 193, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-46-0a8

In [29]:
train_sample.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 106.0 failed 1 times, most recent failure: Lost task 0.0 in stage 106.0 (TID 184, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/Cellar/apache-spark/1.5.2/libexec/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-28-f88